In [ ]:
!pip install snowflake-connector-python pandas
!pip install --upgrade snowflake-connector-python


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
import pandas as pd
import numpy as np

In [1]:
snowflake_options = {
    "account": "XXXXXXXXXXXXXXXX",
    "user": "XXXXXXXXXXXXXXXX",
    "password": "XXXXXXXXXXXXXXXX",
    "database": "XXXXXXXXXXXXXXXX",
    "schema": "XXXXXXXXXXXXXXXX",
    "warehouse": "XXXXXXXXXXXXXXXX",
}

In [ ]:
ctx = snowflake.connector.connect(
    user=snowflake_options['user'],
    password=snowflake_options['password'],
    account=snowflake_options['account'],
    warehouse=snowflake_options['warehouse'],
    database=snowflake_options['database'],
    schema=snowflake_options['schema']
)

account_query = "select * from salesforce_db.raw_data.account"
contact_query = "select * from salesforce_db.raw_data.contact"
opportunity_query = "select * from salesforce_db.raw_data.opportunity"
opportunitystage_query = "select * from salesforce_db.raw_data.opportunitystage"
task_query = "select * from salesforce_db.raw_data.task"

In [ ]:
def get_table(ctx, query):
    cs = ctx.cursor()
    try:
        cs.execute(query)
        df = cs.fetch_pandas_all()
    finally:
        cs.close()
    return df

def get_not_missing_data_columns(df, threshold=50):
    missing_data_percentage = df.isnull().mean() * 100
    filtered_columns = missing_data_percentage[missing_data_percentage <= threshold]
    columns = []
    for column, percentage in filtered_columns.sort_values(ascending=False).items():
        columns.append(column)
    return columns

In [ ]:
account = get_table(ctx, account_query)
opportunity = get_table(ctx, opportunity_query)
contact = get_table(ctx, contact_query)
opportunitystage = get_table(ctx, opportunitystage_query)
task = get_table(ctx, task_query)

In [ ]:
ctx.close()

In [ ]:
account_columns = get_not_missing_data_columns(account, threshold=20)
account_0 = account[account_columns].copy()
account_0.dropna(subset=['HQ_LOCATION__C', 'GROWTH_RATE__C', 'TOTAL_FUNDING_TO_DATE__C', 'OWNER_INTENT_TO_SELL__C'], inplace=True)
columns_to_drop = ['BILLINGLATITUDE', 'BILLINGLONGITUDE', 'BILLINGPOSTALCODE', 'BILLINGCOUNTRY', 'DESCRIPTION', 'TICKERSYMBOL',
                   'BILLINGCITY', 'SITE', 'WEBSITE', 'FAX', 'PHONE', 'BILLINGSTREET', 'SYSTEMMODSTAMP', 'CLEANSTATUS', 'BILLINGSTATE',
                   'LASTMODIFIEDDATE', 'LASTMODIFIEDBYID', 'ISDELETED', 'CREATEDBYID', 'CREATEDDATE', 'OWNERID', 'PHOTOURL']
account_0.drop(columns=columns_to_drop, axis=0, inplace=True)

In [ ]:
opportunity.drop_duplicates(subset=['ID', 'ACCOUNTID'], keep='first', inplace=True)
opportunity_columns = get_not_missing_data_columns(opportunity, threshold=20)
opportunity_0 = opportunity[opportunity_columns].copy()
columns_to_drop = ['ISDELETED', 'FISCAL', 'FISCALQUARTER', 'SYSTEMMODSTAMP', 'LASTMODIFIEDBYID', 'LASTMODIFIEDDATE',
                   'CREATEDBYID', 'CREATEDDATE', 'OWNERID', 'HASOPPORTUNITYLINEITEM', 'HASOPENACTIVITY', 'NEXTSTEP',
                   'CLOSEDATE', 'DESCRIPTION', 'ISPRIVATE']
opportunity_0.drop(columns=columns_to_drop, axis=0, inplace=True)

In [ ]:
contact_columns = get_not_missing_data_columns(contact, threshold=20)
contact_0 = contact[contact_columns].copy()
columns_to_drop = ['HOMEPHONE', 'DESCRIPTION', 'MAILINGCITY', 'MAILINGCOUNTRY', 'MOBILEPHONE', 'BIRTHDATE', 'PHOTOURL', 'ISEMAILBOUNCED',
                   'SYSTEMMODSTAMP', 'LASTMODIFIEDBYID', 'LASTMODIFIEDDATE', 'CREATEDBYID', 'CREATEDDATE', 'OWNERID', 'EMAIL', 'ISDELETED',
                   'FAX', 'PHONE', 'MAILINGSTREET', 'SALUTATION', 'FIRSTNAME', 'LASTNAME', 'CLEANSTATUS']
contact_0.drop(columns=columns_to_drop, axis=0, inplace=True)
values_to_drop = ['Production', 'Technology', 'Procurement', 'Facilities', 'Operations', 'Warehouse Mgmt', 'Administration', 'Executive Team']
contact_0 = contact_0[~contact_0['DEPARTMENT'].isin(values_to_drop)]
values_to_drop = ['VP, Facilities', 'VP, Production', 'VP, Technology', 'SVP, Procurement', 'VP, Finance', 'Dean of Administration', 'Director,Warehouse Mgmt',
                  'SVP, Operations', 'SVP, Technology', 'SVP, Production', 'SVP, Administration and Finance', 'Regional General Manager']
contact_0 = contact_0[~contact_0['TITLE'].isin(values_to_drop)]

In [ ]:
task.drop_duplicates(subset=['ID'], keep='first', inplace=True)
task_columns = get_not_missing_data_columns(task, threshold=20)
task_0 = task[task_columns].copy()
columns_to_drop = ['ISRECURRENCE', 'ISREMINDERSET', 'CALLOBJECT', 'ISARCHIVED', 'SYSTEMMODSTAMP', 'LASTMODIFIEDBYID', 'SUBJECT',
                   'LASTMODIFIEDDATE', 'CREATEDBYID', 'CREATEDDATE', 'ISDELETED', 'DESCRIPTION', 'OWNERID', 'ACTIVITYDATE']
task_0.drop(columns=columns_to_drop, axis=0, inplace=True)

In [ ]:
snowflake_options['schema'] = 'pre_data'
ctx = snowflake.connector.connect(
    user=snowflake_options['user'],
    password=snowflake_options['password'],
    account=snowflake_options['account'],
    warehouse=snowflake_options['warehouse'],
    database=snowflake_options['database'],
    schema=snowflake_options['schema']
)

In [ ]:
account_query = """
    CREATE OR REPLACE TABLE account (
        HQ_LOCATION__C STRING,
        GROWTH_RATE__C FLOAT,
        TOTAL_FUNDING_TO_DATE__C FLOAT,
        YEARSTARTED INT,
        ACCOUNTSOURCE STRING,
        ANNUALREVENUE FLOAT,
        RATING STRING,
        NUMBEROFEMPLOYEES INT,
        OWNERSHIP STRING,
        INDUSTRY STRING,
        TYPE STRING,
        NAME STRING,
        OWNER_INTENT_TO_SELL__C BOOLEAN,
        ID STRING PRIMARY KEY
    )
"""

In [ ]:
opportunity_query = """
    CREATE OR REPLACE TABLE opportunity (
        ID STRING PRIMARY KEY,
        FISCALYEAR INT,
        FORECASTCATEGORYNAME STRING,
        FORECASTCATEGORY STRING,
        ISWON BOOLEAN,
        ISCLOSED BOOLEAN,
        LEADSOURCE STRING,
        TYPE STRING,
        EXPECTEDREVENUE FLOAT,
        PROBABILITY FLOAT,
        AMOUNT FLOAT,
        STAGENAME STRING,
        NAME STRING,
        ACCOUNTID STRING,
        HASOVERDUETASK STRING
    )
"""

In [ ]:
task_query = """
    CREATE OR REPLACE TABLE task (
        ID STRING PRIMARY KEY,
        WHOID STRING,
        CALLDISPOSITION STRING,
        CALLTYPE STRING,
        CALLDURATIONINSECONDS INT,
        ISCLOSED BOOLEAN,
        ACCOUNTID STRING,
        ISHIGHPRIORITY BOOLEAN,
        PRIORITY STRING,
        STATUS STRING,
        WHATID STRING,
        TASKSUBTYPE STRING
    )
"""

In [ ]:
contact_query = """
    CREATE OR REPLACE TABLE contact (
        MAILINGSTATE STRING,
        DEPARTMENT STRING,
        ID STRING PRIMARY KEY,
        TITLE STRING,
        NAME STRING,
        ACCOUNTID STRING
    )
"""

In [ ]:
cursor = ctx.cursor()
table_query = [('account', account_query), ('opportunity', opportunity_query), ('contact', contact_query), ('task', task_query)]
for table, query in table_query:
    cursor.execute(query)

In [ ]:
cursor.close()
ctx.close()

In [ ]:
snowflake_options['schema'] = 'pre_data'
ctx = snowflake.connector.connect(
    user=snowflake_options['user'],
    password=snowflake_options['password'],
    account=snowflake_options['account'],
    warehouse=snowflake_options['warehouse'],
    database=snowflake_options['database'],
    schema=snowflake_options['schema']
)

In [ ]:
def upload_to_snowflake(df, table_name):
    success, nchunks, nrows, _ = write_pandas(ctx, df, table_name)
    return success, nchunks, nrows

In [ ]:
account_0.reset_index(drop=True, inplace=True)
opportunity_0.reset_index(drop=True, inplace=True)
task_0.reset_index(drop=True, inplace=True)
contact_0.reset_index(drop=True, inplace=True)

In [ ]:
upload_to_snowflake(account_0, 'ACCOUNT')
upload_to_snowflake(opportunity_0, 'OPPORTUNITY')
upload_to_snowflake(task_0, 'TASK')
upload_to_snowflake(contact_0, 'CONTACT')

(True, 1, 452)

In [ ]:
ctx.close()